In [1]:
from geoband.API import *
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import geopandas as gpd
import json
from pandas import json_normalize
import folium
from folium.plugins import MarkerCluster
import matplotlib.pyplot as plt
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
from folium import Choropleth
import matplotlib
from datetime import datetime 
# GetCompasData로 주피터 노트북 경로에 파일 저장
from shapely.geometry import Point, Polygon, LineString
from fiona.crs import from_string
matplotlib.rcParams['font.family']="Malgun Gothic"
import pydeck as pdk

### 데이터 불러오기

In [2]:
police_box_4sido=pd.read_csv('치안센터_4개시.csv')
others_one_person = pd.read_csv('33.4개시도_1인가구수.csv')
others_basic_recipient = pd.read_csv('34.4개시도_기초수급자현황.csv')
others_sex_age_pop = pd.read_csv('35.4개시도_성연령별_총인구분포.csv')
others_foreigner_pop = pd.read_csv('36.4개시도_인구데이터(외국인).csv')
others_suicide = pd.read_csv('37.4개시도_자살율.csv')
others_fire = pd.read_csv('38.4개시도_화재발생통계.csv')

### 행정동 경계 geometry 데이터

In [3]:
행정동경계 = gpd.read_file('TL_SCCO_SIG.shp',encoding = 'euc-kr')

In [4]:
행정동_서울 = 행정동경계.iloc[140:165]
행정동_3개시 = 행정동경계[(행정동경계['SIG_KOR_NM'].str.contains('진주'))|행정동경계['SIG_KOR_NM'].str.contains('창원')|행정동경계['SIG_KOR_NM'].str.contains('수원')]

In [5]:
행정동_4개시 =pd.concat([행정동_서울,행정동_3개시]).reset_index(drop = True)

In [6]:
# 지역 이름 매핑
area_list = ['서울']*25 + ['수원']*4 + ['창원']*5 + ['진주']
행정동_4개시['지역'] = area_list

# 구 이름 통일
for i in range(25,34):
    행정동_4개시['SIG_KOR_NM'] = 행정동_4개시['SIG_KOR_NM'].apply(lambda x: x.split(" ")[-1])

행정동_4개시=행정동_4개시[['지역','SIG_KOR_NM','geometry']].rename(columns = {'SIG_KOR_NM':'구'})

### 35.4개시도_성연령별_총인구분포 데이터 매핑
- 청년 15~29
- 중년 30~63
- 고령 64~
- 구별 성별에 따른 총인구

In [7]:
# 구 이름 통일
for i in range(len(others_sex_age_pop)):
    if len(others_sex_age_pop.loc[i,'sgg_nm']) != 2 and len(others_sex_age_pop.loc[i,'sgg_nm']) != 3 and len(others_sex_age_pop.loc[i,'sgg_nm']) != 4:
        others_sex_age_pop.loc[i,'sgg_nm'] = others_sex_age_pop.loc[i,'sgg_nm'][3:]

In [8]:
# 구별 인원수
구별_인구수 = others_sex_age_pop.loc[others_sex_age_pop['age']=='합계']
구별_인구수['popu_num']=구별_인구수['popu_num'].astype(int)
구별_인구수 = 구별_인구수.groupby(['sgg_nm']).agg({'popu_num':'mean'}).reset_index().astype({'popu_num':'int'})

In [9]:
# object 값 int로 변환
others_sex_age_pop=others_sex_age_pop.replace({'X':0})
# 평균연령이 소수가 있어 float 후 int로 변환
others_sex_age_pop['popu_male_num']=others_sex_age_pop['popu_male_num'].astype(float)
others_sex_age_pop['popu_female_num']=others_sex_age_pop['popu_female_num'].astype(float)
others_sex_age_pop['popu_male_num']=others_sex_age_pop['popu_male_num'].astype(int)
others_sex_age_pop['popu_female_num']=others_sex_age_pop['popu_female_num'].astype(int)

In [10]:
청년_list = ['15~19세','20~24세','25~29세']
중년_list = ['30~34세','35~39세','40~44세','45~49세','50~54세','55~59세','60세','61세','62세','63세']
고령_list = ['64세','65세이상']
청년_인구분포 = others_sex_age_pop.loc[others_sex_age_pop['age'].isin(청년_list)]
중년_인구분포 = others_sex_age_pop.loc[others_sex_age_pop['age'].isin(중년_list)]
고령_인구분포 = others_sex_age_pop.loc[others_sex_age_pop['age'].isin(고령_list)]
#각 구의 년도별 평균 인구
청년_인구분포_합계 = 청년_인구분포.groupby(['sgg_nm','date']).agg({'popu_male_num':'sum','popu_female_num':'sum'}).reset_index()
중년_인구분포_합계 = 중년_인구분포.groupby(['sgg_nm','date']).agg({'popu_male_num':'sum','popu_female_num':'sum'}).reset_index()
고령_인구분포_합계 = 고령_인구분포.groupby(['sgg_nm','date']).agg({'popu_male_num':'sum','popu_female_num':'sum'}).reset_index()
# 각 구의 평균 인구
청년_인구분포_평균 = 청년_인구분포_합계.groupby(['sgg_nm']).agg({'popu_male_num':'mean','popu_female_num':'mean'}).reset_index().astype({'popu_male_num':'int','popu_female_num':'int'})
중년_인구분포_평균 = 중년_인구분포_합계.groupby(['sgg_nm']).agg({'popu_male_num':'mean','popu_female_num':'mean'}).reset_index().astype({'popu_male_num':'int','popu_female_num':'int'})
고령_인구분포_평균 = 고령_인구분포_합계.groupby(['sgg_nm']).agg({'popu_male_num':'mean','popu_female_num':'mean'}).reset_index().astype({'popu_male_num':'int','popu_female_num':'int'})
# 컬럼명 변경
청년_인구분포_평균.rename(columns = {'sgg_nm':'구','popu_male_num':'청년_남성인구','popu_female_num':'청년_여성인구'},inplace = True)
중년_인구분포_평균.rename(columns = {'sgg_nm':'구','popu_male_num':'중년_남성인구','popu_female_num':'중년_여성인구'},inplace = True)
고령_인구분포_평균.rename(columns = {'sgg_nm':'구','popu_male_num':'고령_남성인구','popu_female_num':'고령_여성인구'},inplace = True)
# 전체 인구분포 평균 계산
전체_인구분포_평균 = pd.merge(청년_인구분포_평균,중년_인구분포_평균, on = '구')
전체_인구분포_평균 = pd.merge(전체_인구분포_평균,고령_인구분포_평균, on = '구')

In [11]:
행정동_4개시 = pd.merge(행정동_4개시,전체_인구분포_평균, on = '구')

### 4개시도_1인가구수 데이터 매핑
- 성별, 년도별 단독주택/공동주택 관련 feature

In [12]:
others_one_person_1 = others_one_person.copy()

In [13]:
# 구 이름 통일
for i in range(len(others_one_person_1)):
    if len(others_one_person_1.loc[i,'sgg_nm']) != 2 and len(others_one_person_1.loc[i,'sgg_nm']) != 3 and len(others_one_person_1.loc[i,'sgg_nm']) != 4:
        others_one_person_1.loc[i,'sgg_nm'] = others_one_person_1.loc[i,'sgg_nm'][3:]

In [14]:
others_one_person_1=others_one_person_1.replace({'X':0})
others_one_person_1['house_in_commercial'] = others_one_person_1['house_in_commercial'].astype(int)
others_one_person_1['1인가구_공동주택'] = others_one_person_1['single_person_hh'] - others_one_person_1['detached_house'] - others_one_person_1['house_in_commercial']
others_one_person_1.rename(columns= {'sgg_nm':'구','date':'년도','sx':'성별','detached_house':'1인가구_단독주택'},inplace = True)
others_one_person_1 = others_one_person_1[['구','성별','년도','1인가구_단독주택','1인가구_공동주택']]

In [15]:
others_one_person_1_groupby = others_one_person_1.groupby(['구','성별','년도']).agg({'1인가구_단독주택':'sum','1인가구_공동주택':'sum'}).reset_index()
others_one_person_1_groupby = others_one_person_1_groupby.groupby(['구','성별']).agg({'1인가구_단독주택':'mean','1인가구_공동주택':'mean'}).reset_index().astype({'1인가구_단독주택':'int','1인가구_공동주택':'int'})

In [16]:
for i in range(len(others_one_person_1_groupby)):
    for j in range(len(구별_인구수)):
        if others_one_person_1_groupby.loc[i,'구'] == 구별_인구수.loc[j,'sgg_nm']:
                others_one_person_1_groupby.loc[i,'1인가구_공동주택/인구'] = round((others_one_person_1_groupby.loc[i,'1인가구_공동주택']/구별_인구수.loc[j,'popu_num'])*100)
                others_one_person_1_groupby.loc[i,'1인가구_단독주택/인구'] = round((others_one_person_1_groupby.loc[i,'1인가구_단독주택']/구별_인구수.loc[j,'popu_num'])*100)
                       

In [17]:
# 1인가구/인구 count
for i in range(len(others_one_person_1_groupby)):
    for j in range(len(행정동_4개시)):
        if others_one_person_1_groupby.loc[i,'구'] == 행정동_4개시.loc[j,'구']:
            if others_one_person_1_groupby.loc[i,'성별'] =='남자':
                행정동_4개시.loc[j,'남자_공동주택/인구'] = others_one_person_1_groupby.loc[i,'1인가구_공동주택/인구']
                행정동_4개시.loc[j,'남자_단독주택/인구'] = others_one_person_1_groupby.loc[i,'1인가구_단독주택/인구']
            elif others_one_person_1_groupby.loc[i,'성별'] =='여자':
                행정동_4개시.loc[j,'여자_공동주택/인구'] = others_one_person_1_groupby.loc[i,'1인가구_공동주택/인구']
                행정동_4개시.loc[j,'여자_단독주택/인구'] = others_one_person_1_groupby.loc[i,'1인가구_단독주택/인구']

### 4개시도_기초수급자현황 데이터 매핑
- 구와 년도별 기초수급자의 평균 기초수급자의 수를 feature
- 창원시와 진주시, 수원시는 구별로 되어 있지 않아 따로 DF 만들었음

In [18]:
others_basic_recipient = others_basic_recipient[['date','sido','sgg_nm','num']]
for i in range(len(others_basic_recipient)):
    if str(others_basic_recipient['date'][i])[0:4] == '2017': others_basic_recipient['date'][i] = 2017
    elif  str(others_basic_recipient['date'][i])[0:4] == '2018': others_basic_recipient['date'][i] = 2018
    elif  str(others_basic_recipient['date'][i])[0:4] == '2019': others_basic_recipient['date'][i] = 2019
    elif  str(others_basic_recipient['date'][i])[0:4] == '2020': others_basic_recipient['date'][i] = 2020
    elif  str(others_basic_recipient['date'][i])[0:4] == '2021': others_basic_recipient['date'][i] = 2021


In [19]:
기초수급자_4개시_평균 = others_basic_recipient.groupby(['sgg_nm','date']).agg({'num':"mean"}).reset_index().rename(columns = {'num':'평균_기초수급자','date':'년도','sgg_nm':'시군구'})
기초수급자_4개시_평균 = 기초수급자_4개시_평균.groupby(['시군구']).agg({'평균_기초수급자':'mean'}).reset_index()
기초수급자_4개시_평균['평균_기초수급자']=기초수급자_4개시_평균['평균_기초수급자'].astype(int)


### 38.4개시도_화재발생통계 데이터 매핑
- 구별로 연도당 화재발생건수 feature

In [20]:
# 구 이름 통일
for i in range(len(others_fire)):
    if len(others_fire.loc[i,'sgg_nm']) != 2 and len(others_fire.loc[i,'sgg_nm']) != 3 and len(others_fire.loc[i,'sgg_nm']) != 4:
        others_fire.loc[i,'sgg_nm'] = others_fire.loc[i,'sgg_nm'][4:]

In [21]:
others_fire_groupby = others_fire.groupby(['sgg_nm']).agg({'acc_num':'sum'}).reset_index()
#각년도 화재 발생 건수 평균
others_fire_groupby['acc_num'] = round(others_fire_groupby['acc_num']/3)

In [22]:
for i in range(len(others_fire_groupby)):
    for j in range(len(행정동_4개시)):
        if others_fire_groupby.loc[i,'sgg_nm'] == 행정동_4개시.loc[j,'구']:
            행정동_4개시.loc[j,'평균_화재'] = others_fire_groupby.loc[i,'acc_num']

### 36.4개시도_인구데이터(외국인)
- 구별,년도별 외국인 인구 수 feature

In [23]:
# 구 이름 통일
for i in range(len(others_foreigner_pop)):
    if len(others_foreigner_pop.loc[i,'sgg']) != 2 and len(others_foreigner_pop.loc[i,'sgg']) != 3 and len(others_foreigner_pop.loc[i,'sgg']) != 4:
        others_foreigner_pop.loc[i,'sgg'] = others_foreigner_pop.loc[i,'sgg'][4:]

In [24]:
# 2017,2018,2019년 외국인 총인원 count
for i in range(len(others_foreigner_pop)):
    for j in range(len(행정동_4개시)):
        if others_foreigner_pop.loc[i,'sgg'] == 행정동_4개시.loc[j,'구']:
            행정동_4개시.loc[j,'외국인/인구'] = round((others_foreigner_pop.loc[i,'foreigner_all(2017)']+others_foreigner_pop.loc[i,'foreigner_all(2018)']+others_foreigner_pop.loc[i,'foreigner_all(2019)'])/3)

### 37.4개시도_자살율 데이터 매핑
- 연령표준화 사망률: 인구구조가 다른 집단간의 사망 수준을 비교하기 위하여 연령구조가 사망률에 미치는 영향을 제거한 사망률
- 구별로 성별과 연령표준화를 feature

In [25]:
others_suicide=others_suicide[['sgg_nm','sx','age_stndrd_dptt_2017','age_stndrd_dptt_2018','age_stndrd_dptt_2019']]

In [26]:
# 2017,2018,2019년 외국인 총인원 count
for i in range(len(others_suicide)):
    for j in range(len(행정동_4개시)):
        if others_suicide.loc[i,'sgg_nm'] == 행정동_4개시.loc[j,'구']:
            if others_suicide.loc[i,'sx'] =='남자':
                행정동_4개시.loc[j,'평균_남자_자살률'] = round((others_suicide.loc[i,'age_stndrd_dptt_2017']+others_suicide.loc[i,'age_stndrd_dptt_2018']+others_suicide.loc[i,'age_stndrd_dptt_2019'])/3)

            elif others_suicide.loc[i,'sx'] == '여자':
                행정동_4개시.loc[j,'평균_여자_자살률'] = round((others_suicide.loc[i,'age_stndrd_dptt_2017']+others_suicide.loc[i,'age_stndrd_dptt_2018']+others_suicide.loc[i,'age_stndrd_dptt_2019'])/3)
              

In [27]:
행정동_4개시.crs = "EPSG:4326"
행정동_4개시 = 행정동_4개시.to_crs("EPSG:4326")

In [28]:
행정동_4개시

,지역,구,geometry,청년_남성인구,청년_여성인구,중년_남성인구,중년_여성인구,고령_남성인구,고령_여성인구,남자_공동주택/인구,남자_단독주택/인구,여자_공동주택/인구,여자_단독주택/인구,평균_화재,외국인/인구,평균_남자_자살률,평균_여자_자살률
0,서울,종로구,"POLYGON ((956615.453 1953567.199, 956621.579 1...",19229,19855,38874,39624,11398,14552,7.0,7.0,8.0,7.0,234.0,12065.0,33.0,11.0
1,서울,중구,"POLYGON ((957890.386 1952616.746, 957909.908 1...",13114,14538,34883,34625,9365,12187,10.0,5.0,10.0,6.0,224.0,11604.0,25.0,8.0
2,서울,용산구,"POLYGON ((953115.761 1950834.084, 953114.206 1...",21125,22757,61800,61622,15350,20371,7.0,7.0,8.0,8.0,214.0,18545.0,25.0,13.0
3,서울,성동구,"POLYGON ((959681.109 1952649.605, 959842.412 1...",31314,31439,81864,82261,19388,24758,6.0,7.0,7.0,6.0,198.0,11172.0,24.0,10.0
4,서울,광진구,"POLYGON ((964825.058 1952633.250, 964875.565 1...",40674,44016,95220,98700,21791,26608,5.0,9.0,6.0,9.0,195.0,19684.0,26.0,10.0
5,서울,동대문구,"POLYGON ((962141.919 1956519.461, 962149.853 1...",41073,42947,90921,89028,26294,32825,6.0,9.0,7.0,8.0,183.0,19613.0,27.0,10.0
6,서울,중랑구,"POLYGON ((965698.531 1957987.355, 965772.290 1...",36398,36028,107848,107361,29277,35845,4.0,8.0,5.0,7.0,230.0,6563.0,31.0,12.0
7,서울,성북구,"POLYGON ((956787.318 1953542.506, 956761.258 1...",47868,52897,109959,113609,30417,39616,5.0,6.0,6.0,7.0,193.0,14422.0,28.0,12.0
8,서울,강북구,"POLYGON ((956316.789 1965175.345, 956401.551 1...",27991,28826,80988,81825,26003,34222,5.0,7.0,7.0,7.0,185.0,4912.0,29.0,12.0
9,서울,도봉구,"POLYGON ((957223.835 1961199.545, 957195.758 1...",30368,30394,85222,89271,26410,32557,5.0,4.0,7.0,4.0,193.0,2967.0,30.0,13.0
